In [1]:
import sys
import os
import time

atlas_py_path = "C:/Users/natha/OneDrive/Desktop/C++/Atlas/x64/Release"
sys.path.append(atlas_py_path)

import AtlasPy
from AtlasPy.core import Hydra, Portfolio, Strategy
from AtlasPy.ast import *

In [2]:
exchange_id = "test"
portfolio_id = "test_p"
strategy_id = "test_s"
exchange_path_sp500 = "C:/Users/natha/OneDrive/Desktop/C++/Atlas/AtlasTest/scripts/data_sp500.h5"
exchange_path_fast = "C:/Users/natha/OneDrive/Desktop/C++/Atlas/AtlasPy/test/data_fast.h5"
initial_cash = 100.0

ret_sp500 = 2.6207
ret_fast = 0.6351860771192923

hydra = Hydra()

In [3]:
exchange = hydra.addExchange(exchange_id, exchange_path_fast)
portfolio = hydra.addPortfolio(portfolio_id, exchange, initial_cash)
hydra.build()

In [4]:

read_fast = AssetReadNode.make("ma_fast", 0, exchange)
read_slow = AssetReadNode.make("ma_slow", 0, exchange)
spread = AssetDifferenceNode(read_fast, read_slow)
op_variant = AssetOpNodeVariant(spread)

exchange_view = ExchangeViewNode(exchange, op_variant)
exchange_view.setFilter(ExchangeViewFilterType.GREATER_THAN, 0.0)
allocation = AllocationNode(exchange_view)
strategy_node = StrategyNode(allocation, portfolio)
strategy = hydra.addStrategy(Strategy(strategy_id, strategy_node, 1.0))

In [12]:
st = time.perf_counter_ns()
hydra.run()
et = time.perf_counter_ns()

print(f"Time: {(et - st) / 1000} us")

tr = (strategy.getNLV() - initial_cash) / initial_cash
print(f"Total return: {tr:.3%}")
print(f"Epsilon: {tr - ret_fast}")

Time: 96.4 us
Total return: 63.519%
Epsilon: 1.7763568394002505e-15


In [6]:
hydra.reset()